In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['VSV']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('vesicular stomatitis virus', 449), ('vesicular stomatis virus', 4)]

In [9]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [10]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'vesicular stomatis virus': 'Taxonomy:11276',
  'vesicular stomatitis virus': 'Taxonomy:11276'},
 {'Taxonomy:11276': 'Vesicular stomatitis virus'},
 ['Taxonomy:11276']]

In [29]:
grounding_map, names, pos_labels = [{'vesicular stomatis virus': 'Taxonomy:11276',
  'vesicular stomatitis virus': 'Taxonomy:11276'},
 {'Taxonomy:11276': 'Vesicular stomatitis virus'},
 ['Taxonomy:11276']]

In [13]:
excluded_longforms = []

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [15]:
additional_entities = {'HGNC:5413': 'IFITM2'}

In [16]:
unambiguous_agent_texts = {}

In [17]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:5413', 'HGNC:5413', 41)]

In [21]:
intersection2

[('Taxonomy:11276', 'HGNC:5413', 0)]

[('HGNC:7321', ['Musculin', 'musculin'])]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [30]:
names.update(additional_entities)
pos_labels.extend(additional_entities.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-02 02:06:22] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-02 02:06:32] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9448310828844158 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'Taxonomy:11276': 322, 'HGNC:5413': 41},
 'f1': {'mean': 0.944831, 'std': 0.032273},
 'precision': {'mean': 0.946465, 'std': 0.031955},
 'recall': {'mean': 0.950457, 'std': 0.025376},
 'HGNC:5413': {'f1': {'mean': 0.723147, 'std': 0.18725},
  'pr': {'mean': 0.630556, 'std': 0.197515},
  'rc': {'mean': 0.875, 'std': 0.123603}},
 'Taxonomy:11276': {'f1': {'mean': 0.972717, 'std': 0.01357},
  'pr': {'mean': 0.990673, 'std': 0.007616},
  'rc': {'mean': 0.955582, 'std': 0.021467}}}

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [27]:
disamb.dump(model_name, results_path)

In [34]:
print(disamb.info())

Disambiguation model for VSV

Produces the disambiguations:
	IFITM2*	HGNC:5413
	Vesicular stomatitis virus*	Taxonomy:11276

Class level metrics:
--------------------
Grounding                 	Count	F1     
Vesicular stomatitis virus*	322	0.97272
                    IFITM2*	 41	0.72315

Weighted Metrics:
-----------------
	F1 score:	0.94483
	Precision:	0.94646
	Recall:		0.95046

* Positive labels
See Docstring for explanation



In [36]:
preds = disamb.disambiguate(all_texts.values())

In [38]:
a = [text for text, pred in zip(all_texts.values(), preds) if pred[0].startswith('HGNC')]

In [40]:
a

[]